# Perceptron and back propagation

## Some stuffs to begin with


In [134]:
import numpy as np

np.random.seed(0)

In [135]:
def glorot(size):
    limit = np.sqrt(6.0 / sum(size))
    return np.random.uniform(-limit, limit, size)


def prime(fn, x):
    return (fn(x + 1e-8) - fn(x - 1e-8)) / (2 * 1e-8)


def sel(y_hat, y):
    return 0.5 * (y - y_hat) ** 2


def sel_prime(y_hat, y):
    return -(y - y_hat)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_prime(y):
    return y * (1 - y)


def relu(x):
    return x * (x > 0)


def relu_prime(y):
    return 1 * (y > 0)


def tanh(x):
    return np.tanh(x)


def tanh_prime(y):
    return 1 - y**2


def dense(x, w, b):
    return x @ w + b


def dense_grads(x, grad_loss):
    dw = x.T @ grad_loss
    db = np.sum(grad_loss, axis=0, keepdims=True)
    return dw, db


def update_weights(w, b, dw, db, lr):
    w -= lr * dw
    b -= lr * db

## Setup

let $lr$ a constant in [0, 1]

let $x$, $y$ 2 vectors, where:

- $x$ is the input of dimension $(k, n)$
- $y$ is the expected output of dimension $(k, m)$

let $w$, $b$ 2 matrices, where

- $w$ is the weights of the layer of dimension $(n, m)$
- $b$ the bias of the layer of dimension $(1, m)$


In [136]:
x = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1],
    ]
)
y_true = np.array(
    [
        [0],
        [1],
        [1],
        [0],
    ]
)

## Forward pass

We will compute the forward and backward pass of a one dense layer with $n$ inputs and $m$ outputs with an activation function of type $relu$.

**Compute the linear transformation of $x$**

```math
\hat{z} = \hat{x} w + b
```

Then give some non linearity by applying the activation function

```math
\hat{y} = relu(\hat{z}), relu: x \mapsto xH(x)
```

where H is the Heaviside step function

```math
H(x) = \begin{cases}
   1 &\text{if } x > 0 \\
   0 &\text{if } x <= 0
\end{cases}
```

**Compute the $loss$ between the output $\hat{y}$ and the expected output $y$**

```math
loss(\hat{y}, y) = \frac{1}{2}(y - \hat{y})^2 \\
```

## Backward pass

We want to know how much we need to adjust our weights and biases of the layer to minimize the loss function. Basically we need to substract the gradients of the loss along w and b. It is basically a Newton algorithm to find a minima of the forward function minimizing the loss.

**Compute the gradient of the $loss$ along $w$**

```math
\tag{a} \nabla{loss(\hat{y}, y)} \ldotp w = \frac{\partial{loss(\hat{y}, y)}}{\partial{w}}
```

Using the chain rules, we got

```math
\tag{b} \frac{\partial{loss(\hat{y}, y)}}{\partial{w}} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} \frac{\partial{\hat{y}}}{\partial{\hat{z}}} \frac{\partial{\hat{z}}}{\partial{w}}
```

where

```math
\tag{c}  \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} = -(y - \hat{y}) = \hat{y} - y
```

```math
\tag{d}  \frac{\partial{\hat{y}}}{\partial{\hat{z}}} = \frac{\partial{relu(\hat{z})}}{\partial{\hat{z}}} = \frac{\partial{(\hat{z}H(\hat{z}))}}{\partial{\hat{z}}} = 1 * \begin{cases}
   1 &\text{if } \hat{z} > 0 \\
   0 &\text{if } \hat{z} <= 0
\end{cases} = H(\hat{z})
```

```math
\tag{e}  \frac{\partial{(x w + b)}}{\partial{w}} = x^T
```

Combining all together:

```math
\boxed{\nabla{loss(\hat{y}, y)} \ldotp w =  x^T \otimes (\hat{y} - y) H(\hat{z})}
```

**Compute the gradient of the $loss$ along $b$**

```math
\tag{a} \nabla{loss(\hat{y}, y)} \ldotp b = \frac{\partial{loss(\hat{y}, y)}}{\partial{b}}
```

Using the chain rules, we got

```math
\tag{b} \frac{\partial{loss(\hat{y}, y)}}{\partial{b}} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} \frac{\partial{\hat{y}}}{\partial{\hat{z}}} \frac{\partial{\hat{z}}}{\partial{b}}
```

where

```math
\tag{c}  \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} = -(y - \hat{y}) = \hat{y} - y
```

```math
\tag{d}  \frac{\partial{\hat{y}}}{\partial{\hat{z}}} = \frac{\partial{relu(\hat{z})}}{\partial{\hat{z}}} = \frac{\partial{(\hat{z}H(\hat{z}))}}{\partial{\hat{z}}} = 1 * \begin{cases}
   1 &\text{if } \hat{z} > 0 \\
   0 &\text{if } \hat{z} <= 0
\end{cases} = H(\hat{z})
```

```math
\tag{e}  \frac{\partial{(x w + b)}}{\partial{b}} = 1
```

Combining all together:

```math
\boxed{\nabla{loss(\hat{y}, y)} \ldotp b = (\hat{y} - y) H(\hat{z})}
```

**Update the w and b matrices with the respective gradients of the loss.**

```math
\boxed{
   \begin{aligned}
   w = w - lr * \nabla{loss(\hat{y}, y)} \ldotp w \\
   b = b - lr * \nabla{loss(\hat{y}, y)} \ldotp b
   \end{aligned}
}
```

Now, we need to pass the gradient of the loss to the next layer of the neural network and then perform the same calculation as above.

**Compute the gradient of the $loss$ along $\hat{x}$**

```math
\tag{a} \nabla{loss(\hat{y}, y)} \ldotp \hat{x} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{x}}}
```

Using the chain rules, we got

```math
\tag{b} \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{x}}} = \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} \frac{\partial{\hat{y}}}{\partial{\hat{z}}} \frac{\partial{\hat{z}}}{\partial{\hat{x}}}
```

where

```math
\tag{c}  \frac{\partial{loss(\hat{y}, y)}}{\partial{\hat{y}}} = -(y - \hat{y}) = \hat{y} - y
```

```math
\tag{d}  \frac{\partial{\hat{y}}}{\partial{\hat{z}}} = \frac{\partial{relu(\hat{z})}}{\partial{\hat{z}}} = \frac{\partial{(\hat{z}H(\hat{z}))}}{\partial{\hat{z}}} = 1 * \begin{cases}
   1 &\text{if } \hat{z} >= 0 \\
   0 &\text{if } \hat{z} < 0
\end{cases} = H(\hat{z})
```

```math
\tag{e}  \frac{\partial{(x w + b)}}{\partial{x}} = w^T
```

Combining all together:

```math
\boxed{\nabla{loss(\hat{y}, y)} \ldotp \hat{x} = (\hat{y} - y) H(\hat{z}) \otimes w^T}
```

**Finally we can pass the gradient of the loss to the next layer (Hence gradient descent).**


In [191]:
lr = 0.001
w0 = glorot((2, 4))
b0 = np.zeros((1, 4))
w1 = glorot((4, 1))
b1 = np.zeros((1, 1))
loss_fn = lambda x: sel(x, y_true)

for _ in range(100_000):
    # Forward
    z0 = dense(x, w0, b0)
    z1 = tanh(z0)
    z2 = dense(z1, w1, b1)
    y_pred = sigmoid(z2)

    # Compute loss
    loss = np.mean(loss_fn(y_pred))

    # Compute loss gradient along y_pred
    grad_loss = prime(loss_fn, y_pred)

    # Compute loss gradient along the activation function
    grad_loss = grad_loss * prime(sigmoid, z2)

    # Compute the loss gradient along the dense layer
    dw1, db1 = dense_grads(z1, grad_loss)
    grad_loss = grad_loss @ w1.T

    # Compute loss gradient along the activation function
    grad_loss = grad_loss * prime(tanh, z0)

    # Compute the loss gradient along the dense layer
    dw0, db0 = dense_grads(x, grad_loss)
    grad_loss = grad_loss @ w0.T

    update_weights(w1, b1, dw1, db1, lr)
    update_weights(w0, b0, dw0, db0, lr)

display(f"Loss: {loss:.4f}, Predictions: {y_pred.flatten()}")
display(np.all(np.round(y_pred) == y_true))

'Loss: 0.0033, Predictions: [0.04365407 0.91212019 0.91621932 0.09889067]'

np.True_